# Home Credit Default Risk: Data Prep and Combine
In this notebook the data will be prepared and combined for subsequent machine learning. These steps will be as follows:
- Primary data (train and test data sets)
    - Address outliers in the train and test data sets
    - Imput null values in the train and test data sets
    - Get dummy variables on the object data
- bureau and bureau_balance data sets
    - Address outliers in bureau
    - In bureau_balance group by SK_ID_BUREAU and sum
    - Get dummy variables on the object data for both data sets
    - Join bureau_balance to bureau on SK_ID_BUREAU
- previous_application and dependents
    - Address outliers in the previous_application data set these are all in the DAYS... columns.
    - Get dummy variables on the object data.
    - Consolidate the data grouping by SK_ID_CURR and aggregating by sum for object features, and mean for numeric features.
- Join all data sets on SK_ID_CURR

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


# Load the data
train = pd.read_csv('data/application_train.csv', index_col='SK_ID_CURR')
test = pd.read_csv('data/application_test.csv', index_col='SK_ID_CURR')
bureau = pd.read_csv('data/bureau.csv')
bureau_balance = pd.read_csv('data/bureau_balance.csv')
prev_app = pd.read_csv('data/previous_application.csv')
cc_balance = pd.read_csv('data/credit_card_balance.csv')
payments = pd.read_csv('data/installments_payments.csv')
cash_balance = pd.read_csv('data/POS_CASH_balance.csv')

target = train.TARGET
train = train.drop('TARGET', axis='columns')

***
# Preparation of the primary data
## Address outliers
DAYS_EMPLOYED has values at 365243 which is 1000 years

In [2]:
# Find the medain excluding these outliers
day_employed_median = train.loc[train.DAYS_EMPLOYED != 365243, 'DAYS_EMPLOYED'].median()
# Apply this value to the outliers in the train and test data sets
train.loc[train.DAYS_EMPLOYED == 365243, 'DAYS_EMPLOYED'] = day_employed_median
test.loc[test.DAYS_EMPLOYED == 365243, 'DAYS_EMPLOYED'] = day_employed_median

REGION_RATING_CLIENT_W_CITY has a negative value (-1). This is an error.

In [3]:
region_rating_client_w_city_median = train.REGION_RATING_CLIENT_W_CITY.median()
test.loc[test.REGION_RATING_CLIENT_W_CITY == -1, 'REGION_RATING_CLIENT_W_CITY'] = region_rating_client_w_city_median

There are a few instances of categorical data in the training set but not in the test set. I will simply remove these observations.

In [4]:
# Remove the few instances of categorical data in the training set but not in the test set
train.drop(train[train.CODE_GENDER == 'XNA'].index, axis='rows',inplace=True)
train.drop(train[train.NAME_INCOME_TYPE == 'Maternity leave'].index, axis='rows',inplace=True)
train.drop(train[train.NAME_FAMILY_STATUS == 'Unknown'].index, axis='rows', inplace=True)
train.shape

(307500, 120)

## Impute null values
Null values in object columns will be imputed with the string 'no_value'.

In [5]:
obj_columns = train.select_dtypes(include='object').columns

train[obj_columns] = train[obj_columns].fillna('no_value')
test[obj_columns] = test[obj_columns].fillna('no_value')

Null values in the numberic columns will be filled with the median value.

In [6]:
num_columns = train.select_dtypes(exclude='object').columns

for column in num_columns:
    median_value = train[column].median()
    train[column].fillna(median_value, inplace=True)
    test[column].fillna(median_value, inplace=True)

## Get dummy variables

In [7]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# Address any missing columns in each set
train_col_set = set(train.columns)
test_col_set = set(test.columns)
train_missing = list(test_col_set.difference(train_col_set))
test_missing = list(train_col_set.difference(test_col_set))

Only test has missing columns. The columns will be added to the test set and 0 will be broadcast to the columns.

In [8]:
for column in test_missing:
    test[column] = 0
    
# Make sure the column order is the same for train and test
train_columns = train.columns
test = test[train_columns]

***
# Preparation of bureau and bureau_balance

## Address outliers
DAYS_ENDDATED_FACT in bureau has an outlier as -42023. This will be set to the median value

In [9]:
days_enddate_fact_medain = bureau.loc[bureau.DAYS_ENDDATE_FACT != -42023, 'DAYS_ENDDATE_FACT'].median()
bureau.loc[bureau.DAYS_ENDDATE_FACT == -42023, 'DAYS_ENDDATE_FACT'] = days_enddate_fact_medain

## Prepare bureau_balance to combine with bureau
First get dummy variables, then group by SK_ID_BUREAU, finally sum.

In [10]:
bureau_balance = pd.get_dummies(bureau_balance)
bureau_balance = bureau_balance.groupby(['SK_ID_BUREAU']).sum()

# Remove the MONTHS_BALANCE columns
bureau_balance.drop('MONTHS_BALANCE', axis='columns',inplace=True)

## Join bureau and bureau_balance

In [11]:
bureau = bureau.join(bureau_balance, on='SK_ID_BUREAU')

# Remove the 'SK_ID_BUREAU' columns
bureau.drop('SK_ID_BUREAU', axis='columns', inplace=True)

# Get dummies on oject columns
bureau = pd.get_dummies(bureau)

# Group by SK_ID_CURR and sum each column
bureau = bureau.groupby(['SK_ID_CURR']).sum()

***
# Preparation of previous_application and dependents
First drop unneeded columns

In [12]:
prev_app.drop('SK_ID_PREV', axis='columns', inplace=True)
cc_balance.drop('SK_ID_PREV', axis='columns', inplace=True)
payments.drop('SK_ID_PREV', axis='columns', inplace=True)
cash_balance.drop('SK_ID_PREV', axis='columns', inplace=True)

Set the "SK_ID_CURR" column as type int

In [13]:
prev_app['SK_ID_CURR'] = prev_app.SK_ID_CURR.astype('int')
cc_balance['SK_ID_CURR'] = cc_balance.SK_ID_CURR.astype('int')
payments['SK_ID_CURR'] = payments.SK_ID_CURR.astype('int')
cash_balance['SK_ID_CURR'] = cash_balance.SK_ID_CURR.astype('int')

## Address outliers
In the previous_application data set there are many entries listed at 365243. This will be set the median of that column.

In [14]:
# Columns with the outliers
outlier_columns = []
for column in prev_app.columns:
    if (prev_app[column].values == 365243).any():
        print(column)
        outlier_columns.append(column)

# Remove the SK_ID_CURR column. That is not an outlier
outlier_columns = outlier_columns[1:]
        
# Replace the outlier with the median value in each of these columns
for column in outlier_columns:
    median_value = prev_app.loc[prev_app[column] != 365243, column].median()
    prev_app.loc[prev_app[column] == 365243, column] = median_value

SK_ID_CURR
DAYS_FIRST_DRAWING
DAYS_FIRST_DUE
DAYS_LAST_DUE_1ST_VERSION
DAYS_LAST_DUE
DAYS_TERMINATION


## previous_application, get dummies and consolidate
The data must be consolidated to so the values of SK_ID_CURR are unique. To do this I will us .groupby and an aggregate function. For the object data, I will aggregate by sum. For the numeric data, I will aggregate by mean.

In [15]:
# Make objects df with the sum returned
prev_app_objects_columns = list(prev_app.loc[:,prev_app.dtypes == 'object'].columns)
prev_app_objects_columns.append('SK_ID_CURR')
prev_app_objects = prev_app.loc[:,prev_app_objects_columns]
prev_app_objects_dummies = pd.get_dummies(prev_app_objects)
prev_app_objects_sum = prev_app_objects_dummies.groupby(['SK_ID_CURR']).sum()

In [16]:
# Make numbers df with the mean returned
prev_app_numbers_columns = list(prev_app.loc[:,prev_app.dtypes != 'object'].columns)
prev_app_numbers = prev_app.loc[:,prev_app_numbers_columns]
prev_app_numbers_mean = prev_app_numbers.groupby(['SK_ID_CURR']).mean()

In [17]:
# recombine the object and numeric dataframes
prev_app = prev_app_numbers_mean.join(prev_app_objects_sum)
prev_app.shape

(338857, 162)

## cash_balance, get dummies and consolidate

In [18]:
# Get dummies and group by SK_ID_CURR
cash_balance_dummies = pd.get_dummies(cash_balance)
cash_balance = cash_balance_dummies.groupby(['SK_ID_CURR']).sum()

## payment, consolidation
I will create three features from this data. 
- The difference in days between installment and payment -> groupby SK_ID_CURR and summed
- The difference in ammount between installment and payment -> groupby SK_ID_CURR and summed
- The number of unique installment versions when grouped by SK_ID_CURR

In [19]:
# make DAYS_DELTA column
payments['DAYS_DELTA'] = payments.DAYS_ENTRY_PAYMENT - payments.DAYS_INSTALMENT

# make AMT_DELTA column
payments['AMT_DELTA'] = payments.AMT_INSTALMENT - payments.AMT_PAYMENT

# make a df for the sum of DAYS_DELTA and AMT_DELTA
payment_delta = payments[['SK_ID_CURR','DAYS_DELTA','AMT_DELTA']]

# make new df of the SUM
payment_delta_sum = payment_delta.groupby(['SK_ID_CURR']).sum()

Below are the number of unique installment versions.

In [20]:
# make a separate df to count the number of unique payment versions
payment_version = payments[['SK_ID_CURR','NUM_INSTALMENT_VERSION']]

# make new df of the unique counts
payment_version_unique = payment_version.groupby(['SK_ID_CURR']).nunique()

# rename the NUM_INSTALMENT_VERSION columsn
payment_version_unique.rename({"NUM_INSTALMENT_VERSION":"NUM_INSTALMENT_UNIQUE"},
                              axis='columns', inplace=True)

The three variables are then combine to a new payment dataframe

In [21]:
# Join payment_version_unique and payment_delta data frame
payment = payment_version_unique.join(payment_delta_sum)

## cc_balance, consolidated
With the cc_balance data set, I will create a new feature for holding an balance over the credit limit. This will be called 'AMT_OVER_LIMIT', and cases not over the limit will be set to 0.<p>
Then I will group the object features by SK_ID_CURR and sum, and similarly group the numeric features by SK_ID_CURR and aggregate by the mean.

In [22]:
# Feature engineering make column for balance over the credit limit
cc_balance['AMT_OVER_LIMIT'] = cc_balance.AMT_BALANCE - cc_balance.AMT_CREDIT_LIMIT_ACTUAL

# Set any negative values to 0
cc_balance.loc[cc_balance.AMT_OVER_LIMIT <0, 'AMT_OVER_LIMIT'] = 0

In [23]:
# Make objects df with the dummy variables summed
cc_balance_objects = cc_balance.loc[:,['SK_ID_CURR','NAME_CONTRACT_STATUS']]
cc_balance_objects_dummies = pd.get_dummies(cc_balance_objects)
cc_balance_objects_sum = cc_balance_objects_dummies.groupby(['SK_ID_CURR']).sum()

In [24]:
# Make numbers df with the mean returned 
cc_balance_numbers = cc_balance.drop(['NAME_CONTRACT_STATUS'], axis='columns')
cc_balance_numbers_dummies = pd.get_dummies(cc_balance_numbers)
cc_balance_numbers_mean = cc_balance_numbers_dummies.groupby(['SK_ID_CURR']).mean()

In [25]:
# join cc_balance_objects_sum and cc_balance_numbers_sum
cc_balance = cc_balance_numbers_mean.join(cc_balance_objects_sum)

In [26]:
print('train',train.shape)
print('test',test.shape)
print('bureau',bureau.shape)
print('prev_app',prev_app.shape)
print('cash_balance',cash_balance.shape)
print('payments',payments.shape)
print('cc_balance',cc_balance.shape)

train (307500, 247)
test (48744, 247)
bureau (305811, 43)
prev_app (338857, 162)
cash_balance (337252, 14)
payments (13605401, 9)
cc_balance (103558, 28)


## Rename columns that will collide

In [27]:
bureau.rename({'AMT_ANNUITY': 'AMT_ANNUITY_bureau'}, axis='columns', inplace=True)

In [28]:
for column in prev_app.columns:
    new_column_name = column + '_prev_app'
    prev_app.rename({column: new_column_name}, axis='columns', inplace=True)

In [29]:
for column in cash_balance.columns:
    new_column_name = column + '_cash_balance'
    cash_balance.rename({column: new_column_name}, axis='columns', inplace=True)

***
# Join data sets
Expected number of columns after joining all sets

In [30]:
train.shape[1] + bureau.shape[1] + prev_app.shape[1] + cc_balance.shape[1] + payments.shape[1] + cash_balance.shape[1]

503

Form final training set

In [31]:
train_all = train.join(bureau).join(prev_app).join(cc_balance).join(payments).join(cash_balance)
train_all.shape

(307500, 503)

Form final test set

In [32]:
test_all = test.join(bureau).join(prev_app).join(cc_balance).join(payments).join(cash_balance)
test_all.shape

(48744, 503)

## Export data as a csv file

In [33]:
train_all.to_csv('train_all.csv')
test_all.to_csv('test_all.csv')